<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/Dialogue_Speaker_Extract_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.4-py3-none-any.whl size=12316 sha256=cfcff548b40c7da4125dfb11e1ce20f0348d1942d686a5177d7f559fee538400
  Stored in directory: /root/.cache/pip/wheels/47/1d/3c/92d88493ed40c0d9be60

可能这里需要添加下HF_TOKEN

In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract_qwen18", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("silk-road/Haruhi-Dialogue-Speaker-Extract_qwen18", device_map="auto", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract_qwen18:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/64.0 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract_qwen18:
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen_generation_utils.py:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract_qwen18:
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Dialogue-Speaker-Extract_qwen18:
- modeling_qwen.py
- cpp_kernels.py
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/250 [00:00<?, ?B/s]

In [3]:
sys_prompt = "给定input paragraph，抽取其中的对话，并输出为json格式 Let's think it step by step 1. summarize input paragraph into bullet format，存储在summary字段 2. 抽取每一句对话的内容 dialogue，判断每一句话的说话人 said by, 存储在conversations中\n"

In [4]:
test_para_0 = "夏子明要求单翔重复之前说过的话，然后要求黎明给他做饭，最终吃了两份饭菜，引发了关于打打杀杀生活选择的对话。夏子明最终以吃了两碗饭为胜利理由，要求黎明认输。\n“可是我吃的是你做的饭，我并不亏，因为要是我去做饭的话，不但会浪费灵力，做出来的有可能还没这么美味”夏子明道。\n黎明摇摇头“这并不是美味，你身后的人做出来的饭菜那才是真正的美味，有兴趣一起尝尝吗”\n气氛，刹那之间便冷了下来。\n夏子明目光直视黎明，一手却是提起了身边的单翔，血腥的笑意溢于言表“其实，想要得到五彩石的方法还有一个，那便是直接将七窍玲珑心掏出来，装在我的身体里便是”\n“你敢”两个字脱口而出。\n这次换做夏子明冷笑了“我有何不敢”\n确实，他确实没什么不敢的，单翔与他非亲非故，对于他来说，本来就是一个用来激发五彩石力量的工具而已，他为什么不敢？他本来就是应该这样做的。\n而他却没有什么能够威慑到他的存在，以自己现在的逆天之力，他或许能够和他一战，可一开战的话，他就势必会用五彩石，从而加快他动作的节奏。然唯一能够对他形成的威慑的人，在这一刻却是不会出手，而且就算他们陷入危机，沉静出手，她救的只会是他，不会是单翔，且他也不会动手杀夏子明。\n所以，眼下的事情还是得要靠他解决。\n黎明的眉头微微蹙起，若有所思道“确实，你确实没什么不敢的，你就当我随口说说罢了，不必较真”\n夏子明一边解单翔的衣服，一边说道“若是你还有什么话没说完的话，可以继续说，我也可以边挖边听的。只是我要提醒你一点，你说得越快，我挖得越快，他就会少一些痛苦。若你说得慢了，那我便会放下速度，他也会多一些痛苦”\n黎明微怒道“你怎么不按套路出牌”\n夏子明微微一笑，看来已经找到了那颗七窍玲珑心的准确位置，准备下手。"

In [5]:
model = model.eval()
response, history = model.chat(tokenizer, sys_prompt + test_para_0, history=[])
print(response)


{"summary": "- 夏子明要求单翔重复之前说过的话，然后要求黎明给他做饭，最终吃了两份饭菜，引发了关于打打杀杀生活选择的对话。夏子明最终以吃了两碗饭为胜利理由，要求黎明认输。", "conversations": [{"dialogue": "可是我吃的是你做的饭，我并不亏，因为要是我去做饭的话，不但会浪费灵力，做出来的有可能还没这么美味", "said_by": "夏子明"}, {"dialogue": "这并不是美味，你身后的人做出来的饭菜那才是真正的美味，有兴趣一起尝尝吗", "said_by": "黎明"}, {"dialogue": "其实，想要得到五彩石的方法还有一个，那便是直接将七窍玲珑心掏出来，装在我的身体里便是", "said_by": "夏子明"}, {"dialogue": "你敢", "said_by": "黎明"}, {"dialogue": "我有何不敢", "said_by": "夏子明"}, {"dialogue": "果然是个十成中的十成能手！", "said_by": "夏子明"}]}


In [7]:
test_para_1 = "隐藏在丛林身后，看着年轻男子从怀中掏出一个红色的瓶子，然后将其打开，从中放出一大群一尺来长的尸虫，紧接着他又把一个篮子瓶子拿出来，从里面取出一男一女两具尸体，看那男女的样子，像是正在交、欢的时候被杀死的。\n触目惊心的一幕发生了！\n当那些尸虫见到那两具尸体的时候，立马从其口鼻耳洞中钻了进去，几个呼吸的时间，那两具尸体像是泄气的气球顿时塌陷了下去。当那些尸虫再从其口鼻耳中爬出来的时候，已然有了一条蛇那么大。\n黎明听到那男子在喃喃自语道“这人皮和骨骼都能够卖个好价钱，可不能浪费了”说着又从怀里拿出一个白色的瓶子将那两具只剩下人皮和骨骼的尸体给装了进去。\n那些尸虫好似听懂了男子话，飞快分散在四面八方，很快隐匿在丛林间，消失在黎明二人的视线。\n做完这一切，男子一手捂着嘴，长长地打了一个哈欠，走到旁边的一草丛，躺下便开始酣睡。\n黎明和沉静面面相觑，没有进行任何的交流，但都看出了对方的心思，那便是等，看这男子会玩出个什么样的花样来。\n时间并没花上多久，大概就过了一刻钟左右的样子，一条尸虫爬了回来。\n它先是用尾巴轻轻拍了拍男子，可男子刚刚睡着，这段时间，是最忌被人打扰的时间段，因此翻了个身，没有理会它，继续睡。\n尸虫又等了一会儿，见男子没有醒来，它没有因此而产生烦恼，只是看着眼前那一块块新鲜的肉，咧着嘴止不住地流口水。\n突然，像是下定了某种决心一样，它猛地张开大口，一口咬在男子的手臂上。\n尸虫像是犯错的孩子，低沉着头，一副委屈的样子。\n尸虫一阵兴奋，迅速向丛林中蹿去，男子紧跟其后。\n当黎明纳闷，为何这男子不等其他尸虫回来，便开始独自行动的时候，赫然发现，其余的尸虫全都包裹在了他俩身边。"
response, history = model.chat(tokenizer, sys_prompt + test_para_1, history=[])
print(response)


{"summary": "- 男子展示了神秘的能力，放生了大量的尸虫和尸体，引发了一系列惊奇的情节和互动。\n- 夜晚男子在树林中遇到了一只受伤的尸虫，它请求食物并将被束之高阁的命运变成了一场冒险。", "conversations": [{"dialogue": "这人皮和骨骼都能够卖个好价钱，可不能浪费了", "said_by": "男子"}, {"dialogue": "今晚一起睡觉吧", "said_by": "黎明"}]}
